<a href="https://colab.research.google.com/github/PankajSingh013/Deep_learning/blob/main/Gradient_Descent_and_NN_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np


In [4]:
df = pd.read_csv('insurance.csv')
df.head()

,age,affordibility,bought_insurance,Unnamed: 3
0,22,1,0,NaN
1,25,0,0,NaN
2,47,1,1,NaN
3,52,0,0,NaN
4,46,1,1,NaN


In [5]:
df = df.drop(columns='Unnamed: 3',axis=1)

In [18]:
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [6]:
#train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df[['age','affordibility']],df['bought_insurance'],test_size=0.2,random_state=2)

In [25]:
X_train.shape,X_test.shape

((22, 2), (6, 2))

In [26]:
y_train.shape

(22,)

In [7]:
#scaling the age column
x_train_scaled = X_train.copy()
x_train_scaled['age'] = x_train_scaled['age']/100
x_test_scaled = X_test.copy()
x_test_scaled['age'] = x_test_scaled['age']/100

In [35]:
type(X_train),y_train.shape

(pandas.core.frame.DataFrame, (22,))

In [8]:
from tensorflow.python.distribute.cluster_resolver import kubernetes_cluster_resolver
#simple NN using tf
model= keras.Sequential([
    keras.layers.Dense(1,input_shape = (2,),kernel_initializer='ones',bias_initializer='zeros',activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train_scaled,y_train,epochs=5000)

Streaming output truncated to the last 5000 lines.
Epoch 1548/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5792 - accuracy: 0.6818
Epoch 1610/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5791 - accuracy: 0.6818
Epoch 1611/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5791 - accuracy: 0.6818
Epoch 1612/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5790 - accuracy: 0.6818
Epoch 1613/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5790 - accuracy: 0.6818
Epoch 1614/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5789 - accuracy: 0.6818
Epoch 1615/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5789 - accuracy: 0.6818
Epoch 1616/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5789 - accuracy: 0.6818
Epoch 1617/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5788 - accuracy: 0.6818
Epoch 1618/5000
1/1 [==================

In [9]:
model.evaluate(x_test_scaled,y_test)

1/1 [==============================] - 0s 416ms/step - loss: 0.3153 - accuracy: 1.0000


[0.3152588903903961, 1.0]

In [10]:
#accuracy using tensorflow is 100% for our sample dataset
#to know the calculated weights
coef,intercept=model.get_weights()

In [11]:
coef,intercept

(array([[4.919666 ],
        [1.4286091]], dtype=float32), array([-2.8673074], dtype=float32))

In [12]:
#implementing predict from scratch
def sigmoid(x):
    return 1/(1+np.exp(-x))
sigmoid(18)

0.9999999847700205

In [15]:
def prediction_fn(age,affordibility):
    return sigmoid(coef[0]*age + coef[1]*affordibility + intercept)

prediction_fn(.47,1)


array([0.7054828], dtype=float32)

In [16]:
def logloss(y,y_predicted):
   epsilon = 1e-15 
   ypr = [max(epsilon,i)for i in y_predicted]
   ypr = np.array([min(i,1-epsilon) for i in ypr])
   #print(ypr)
   logloss= -np.mean(y*np.log(ypr) + (1-y)*np.log(1-ypr))
   return logloss

In [31]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid(weighted_sum)
        loss = logloss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        if i%50==0:
            print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            break

    return w1, w2, bias


In [19]:
df.columns

Index(['age', 'affordibility', 'bought_insurance'], dtype='object')

In [32]:
gradient_descent(x_train_scaled['age'],x_train_scaled['affordibility'],y_train,1000,0.4804 )

Epoch:0, w1:0.973391914875179, w2:0.9484790550524685, bias:-0.1133436520447742, loss:0.714610156590683
Epoch:50, w1:1.434118908813528, w2:1.1205902839299346, bias:-1.213046034118164, loss:0.5746014191876968
Epoch:100, w1:2.079276479842114, w2:1.3122169122569924, bias:-1.6236034220276634, loss:0.5494269324988902
Epoch:150, w1:2.6881856069465204, w2:1.3899807229711387, bias:-1.9333767096348242, loss:0.5304146897215756
Epoch:200, w1:3.251444113258653, w2:1.4247457568978772, bias:-2.1914685336563027, loss:0.5149521957697487
Epoch:250, w1:3.7684786837804563, w2:1.4435867245385698, bias:-2.4182835285402966, loss:0.502142692070128
Epoch:300, w1:4.2420074958184, w2:1.457067938810731, bias:-2.6230993640616744, loss:0.4914526974026106
Epoch:350, w1:4.675802877729502, w2:1.4692197476438582, bias:-2.810613490780118, loss:0.4824843559351839
Epoch:363, w1:4.782566758698436, w2:1.472366470447292, bias:-2.8568944256084516, loss:0.480394278092264


(4.782566758698436, 1.472366470447292, -2.8568944256084516)

In [33]:
model.get_weights()

[array([[4.919666 ],
        [1.4286091]], dtype=float32), array([-2.8673074], dtype=float32)]

In [58]:
#implementing NN class
class NN :
    def __init__(self):
        self.w1=1
        self.w2=1
        self.bias=0
    
    def predict(self,x):
        weighted_sum=self.w1*x['age'] + self.w2*x['affordibility']+self.bias
        return sigmoid(weighted_sum)
    def fit(self,x,y,epochs,loss_threshold):
        self.w1,self.w2,self.bias= self.gradient_descent(x['age'],x['affordibility'],y,epochs,loss_threshold)

    def gradient_descent(self,age, affordibility, y_true, epochs, loss_thresold):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordibility + bias
            y_predicted = sigmoid(weighted_sum)
            loss = logloss(y_true, y_predicted)

            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted-y_true)) 

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d

            print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

            if loss<=loss_thresold:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias

        

In [59]:
nn = NN()
nn.fit(x_train_scaled,y_train,epochs=5,loss_threshold=0.48039)

Epoch:0, w1:0.973391914875179, w2:0.9484790550524685, bias:-0.1133436520447742, loss:0.714610156590683
Epoch:1, w1:0.9526050112606814, w2:0.9061888855737805, bias:-0.21202325373024944, loss:0.6844545754122794
Epoch:2, w1:0.9371389216394038, w2:0.8724863718378795, bias:-0.297354777348674, loss:0.6623114062843333
Epoch:3, w1:0.9263972300850559, w2:0.8464964034855741, bias:-0.3708685783767047, loss:0.6463299135248587
Epoch:4, w1:0.9197530021886515, w2:0.8272363836850969, bias:-0.4341501414207025, loss:0.6349055537023248


In [60]:
nn.predict(x_test_scaled)

1     0.449124
0     0.644611
14    0.699258
9     0.721946
19    0.636138
17    0.716373
dtype: float64